In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model_name = "AbdulmohsenA/Faseeh"
teacher_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

d:\anaconda\envs\Faseeh\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--AbdulmohsenA--Faseeh. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/217 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.1k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/4.23k [00:00<?, ?B/s]

In [4]:
teacher_model.model

M2M100Model(
  (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
  (encoder): M2M100Encoder(
    (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (embed_positions): M2M100SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-11): 12 x M2M100EncoderLayer(
        (self_attn): M2M100Attention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (activation_fn): ReLU()
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)

In [39]:
# Pruning the vocab size

def get_valid_tokens(tokenizer):
    valid_tokens = set()  # Use a set to avoid duplicates
    vocab = tokenizer.get_vocab()

    for token, index in vocab.items():
        # Check if the token contains Arabic characters
        if any('\u0600' <= char <= '\u06FF' for char in token):
            valid_tokens.add(index)

    return valid_tokens

# Get valid tokens for Arabic and English
valid_tokens = get_valid_tokens(tokenizer)
print(f"Number of valid tokens: {len(valid_tokens)}")
vocab = {t: v for v, t in tokenizer.get_vocab().items() if t in valid_tokens}

Number of valid tokens: 14168


In [48]:
# TEST FOLLOW THROUGH
sample = tokenizer("This is a test", return_tensors='pt')
print("Tokenized:", sample)

Tokenized: {'input_ids': tensor([[256047,   9680,    248,      9,   7356,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [81]:
embedding = teacher_model.get_encoder()(**sample)


In [82]:
print(embedding['last_hidden_state'].shape)

torch.Size([1, 6, 1024])


In [92]:
def print_shape_hook(module, input, output):
    print(f"Layer: {module.__class__.__name__}, Output Shape: {output.shape}")


In [94]:
# Register hooks for the encoder layers
for layer in teacher_model.encoder.layers:
    layer.register_forward_hook(print_shape_hook)

# Register hooks for the decoder layers (if applicable)
for layer in teacher_model.decoder.layers:
    layer.register_forward_hook(print_shape_hook)


AttributeError: 'M2M100ForConditionalGeneration' object has no attribute 'encoder'

In [ ]:
# Example input
input_text = "Hello, how are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Run the model
with torch.no_grad():
    outputs = model(input_ids)
